In [2]:
import pandas as pd
import scipy.stats

# Calculate Ripeness Probability and Harvesting Window

In [7]:
# Read the CSV file
data = pd.read_csv('/Users/claudiado/Documents/falling fruit/harvesting_window/all_species_model_outputs_ripeness_curves.csv')

# Calculate probabilities of ripeness for every day of the year
def calculate_ripeness_probability(mean_ripeness, sd_ripeness, day):
    if sd_ripeness == 0:
        return 0  # Avoid division by zero
    z_score = (day - mean_ripeness) / sd_ripeness
    probability = scipy.stats.norm.cdf(z_score)
    return probability

# Create a range of days (e.g., 1 to 365) representing the days of the year
days_of_year = range(1, 366)

# Loop through each day and calculate probabilities
for day in days_of_year:
    data[f'ripeness_probability_day_{day}'] = data.apply(lambda row: calculate_ripeness_probability(row['mean_ripeness'], row['sd_ripeness'], day), axis=1)

# Calculate harvesting windows for species with sd = 0
harvesting_window_days = 30  # Adjust this value as needed
data.loc[data['sd_ripeness'] == 0, 'harvesting_window_start'] = data['mean_ripeness'] - harvesting_window_days
data.loc[data['sd_ripeness'] == 0, 'harvesting_window_end'] = data['mean_ripeness'] + harvesting_window_days

# Export the updated data to a new CSV file
data.to_csv('/Users/claudiado/Documents/falling fruit/harvesting_window/calculated_ripeness_and_harvesting_windows.csv', index=False)
print(data)


    Unnamed: 0                          species  mean_ripeness  sd_ripeness  \
0            0               Rubus occidentalis          186.2          9.4   
1            1                     Ficus carica          206.8         13.1   
2            2                    Olea europaea          197.9         15.5   
3            3                     Olea europea          300.5         11.8   
4            4                      Morus rubra          185.6          8.7   
..         ...                              ...            ...          ...   
70          70  Pyrus communis 'Early cultivar'          245.0          0.0   
71          71   Pyrus communis 'Late cultivar'          273.8          5.8   
72          72        Pyrus communis 'Williams'          270.0         11.2   
73          73        Pyrus communis 'Bartlett'          245.0          0.0   
74          74                  Pyrus pyrifolia          245.0          0.0   

   formatted_ripeness_date  60_day_ripeness_end  30

# Merge Harvesting Curve with Falling Fruit Observation Data

In [9]:
# Read the CSV files
data = pd.read_csv('/Users/claudiado/Documents/falling fruit/harvesting_window/calculated_ripeness_and_harvesting_windows.csv')
observations = pd.read_csv('/Users/claudiado/Documents/falling fruit/harvesting_window/observation_phenology.csv')

# Merge the dataframes based on the 'scientific_name' column
merged_data = pd.merge(data, observations, on='scientific_name', how='inner')

# Export the merged data to a new CSV file
merged_data.to_csv('/Users/claudiado/Documents/falling fruit/harvesting_window/ff_harvesting_curve.csv', index=False)
print(merged_data)


      Unnamed: 0     scientific_name  mean_ripeness  sd_ripeness  \
0              0  Rubus occidentalis          186.2          9.4   
1              0  Rubus occidentalis          186.2          9.4   
2              0  Rubus occidentalis          186.2          9.4   
3              0  Rubus occidentalis          186.2          9.4   
4              0  Rubus occidentalis          186.2          9.4   
...          ...                 ...            ...          ...   
5720          74     Pyrus pyrifolia          245.0          0.0   
5721          74     Pyrus pyrifolia          245.0          0.0   
5722          74     Pyrus pyrifolia          245.0          0.0   
5723          74     Pyrus pyrifolia          245.0          0.0   
5724          74     Pyrus pyrifolia          245.0          0.0   

     formatted_ripeness_date  60_day_ripeness_end  30_day_ripeness_end  \
0                      5-Jul                246.2                216.2   
1                      5-Jul       

# Determine ripeness probability given scientific_name, location_id, doy

In [16]:
# Read the CSV file
data = pd.read_csv('/Users/claudiado/Documents/falling fruit/harvesting_curve/ff_harvesting_curve.csv')

# User input for species scientific_name, lat, lng, and day of year
scientific_name = input('Enter scientific name: ')
latitude = float(input('Enter latitude: '))
longitude = float(input('Enter longitude: '))
day_of_year = int(input('Enter day of the year: '))
    

def get_ripeness():    
    # Filter the DataFrame based on scientific_name, lat, and lng
    filtered_data = data[(data['scientific_name'] == scientific_name) & (data['lat'] == latitude) & (data['lng'] == longitude)]
    
    if len(filtered_data) == 0:
        return None  # scientific_name/Location not found
    
    # Get the ripeness probability for the specified day of the year
    ripeness_probability_col = f'ripeness_probability_day_{day_of_year}'
    ripeness_probability = filtered_data.iloc[0][ripeness_probability_col]
    
    return ripeness_probability

# Call the function and display the result
probability = get_ripeness()
if probability is not None:
    print(f'Ripeness Probability for {scientific_name} on day {day_of_year}: {ripeness_prob}')
else:
    print(f'No data found for {scientific_name}')
    

Enter scientific name: Rubus occidentalis
Enter latitude: 44.62997331
Enter longitude: -88.48640119
Enter day of the year: 186
No data found for Rubus occidentalis
